In [1]:
import datetime
import os.path


In [2]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
CREDENTIALS_FILE = "client-secret.json"
TOKEN_FILE = "token.json"
SCOPES = ["https://www.googleapis.com/auth/calendar"]

In [4]:
def create_event(service, summary, start_time, end_time, description= "", location= ""):
    event = {
        "summary": summary,
        "location": location,
        "description": description,
        "start": {
            "dateTime": start_time.isoformat(),
            "timeZone": "America/Los_Angeles",
        },
        "end": {
            "dateTime": end_time.isoformat(),
            "timeZone": "America/Los_Angeles",
        },
        "reminders":{
            "useDefault": False,
            "overrides": [
                {
                    "method": "email",
                    "minutes": 10
                },
                {
                    "method": "popup",
                    "minutes": 10
                }
            ]
        }
    }

    try:
        event = service.events().insert(calendarId="primary", body=event).execute()
        print(f"Event created: {event.get('htmlLink')}")
        return event
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

In [1]:
def get_time():
    return datetime.datetime.now(datetime.timezone.utc).isoformat()

In [5]:
def get_events(service):
    now = datetime.datetime.now(datetime.timezone.utc).isoformat()
    print("getting events")
    events_result = (
        service.events()
        .list(
            calendarId='primary',
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy='startTime'
        )
        .execute()
    )
    events = events_result.get('items', [])

    if not events:
        print("no events found.")
    else:
        print("Upcoming events:")
        for event in events:
            start = event["start"].get("dateTime")
            if start is None:
                start = event["start"].get("date")
                
            print(f"  {start} - {event["summary"]}")



In [6]:
def test_event(service):
    now = datetime.datetime.now(datetime.timezone.utc)
    print(now)
    start_time = now + datetime.timedelta(days=1)
    end_time = start_time + datetime.timedelta(hours=1)

    return create_event(
        service,
        summary="Test Event",
        start_time=start_time,
        end_time=end_time,
        description="This is a test event",
        location="Virtual"
    )

In [7]:
def create_calendar_client():
    
    creds = None

    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    

    try:
        service = build("calendar", "v3", credentials = creds)
        return service

    except HttpError as error:
        print(f"An error occurred: {error}")

In [8]:
calendar_client = create_calendar_client()

test_event(calendar_client)



2025-08-08 04:09:35.577995+00:00
Event created: https://www.google.com/calendar/event?eid=YmV0cGQwYmYzOTIyN2pxZ2xwOTBzYW5maG8gc3llc3NzMTMyQG0


{'kind': 'calendar#event',
 'etag': '"3509252360828510"',
 'id': 'betpd0bf39227jqglp90sanfho',
 'status': 'confirmed',
 'htmlLink': 'https://www.google.com/calendar/event?eid=YmV0cGQwYmYzOTIyN2pxZ2xwOTBzYW5maG8gc3llc3NzMTMyQG0',
 'created': '2025-08-08T04:09:40.000Z',
 'updated': '2025-08-08T04:09:40.414Z',
 'summary': 'Test Event',
 'description': 'This is a test event',
 'location': 'Virtual',
 'creator': {'email': 'syesss132@gmail.com', 'self': True},
 'organizer': {'email': 'syesss132@gmail.com', 'self': True},
 'start': {'dateTime': '2025-08-08T21:09:35-07:00',
  'timeZone': 'America/Los_Angeles'},
 'end': {'dateTime': '2025-08-08T22:09:35-07:00',
  'timeZone': 'America/Los_Angeles'},
 'iCalUID': 'betpd0bf39227jqglp90sanfho@google.com',
 'sequence': 0,
 'reminders': {'useDefault': False,
  'overrides': [{'method': 'email', 'minutes': 10},
   {'method': 'popup', 'minutes': 10}]},
 'eventType': 'default'}

In [9]:
get_events(calendar_client)

getting events
Upcoming events:
  2025-08-08T21:09:35-07:00 - Test Event
